# Data Cleaning for the scraped data

### Necessary imports

In [6]:
import pandas as pd # Pandas library

# Importing the data

In [63]:
df = pd.read_csv("dataframe.csv")
df.head()

,dataset_name,url,摘要,開放條件,數據分類,數據格式,提供部門/實體,提供部門/實體聯絡電話,提供部門/實體聯絡電郵,更新頻率,...,字段描述（葡萄牙文）,has_api,API名稱,請求方式,返回類型,調用地址,接口描述,api_key,api_val,api_summary
0,酒吧資料,https://data.gov.mo/Detail?id=b4fa6540-4295-4b...,已領有牌照之實體 - 酒吧資料,完全開放,旅遊及博彩,XML,旅遊局,(853)28315566,webmaster@macaotourism.gov.mo,每日,...,"['Nome em T.chinês', 'Nome em S.chinês', 'Nome...",True,dst_bars_list,GET,XML,https://dst.apigateway.data.gov.mo/dst_bars,已領有牌照之實體 - 酒吧 Licensed Entities - Bars 已领有牌照之实...,Authorization,APPCODE 09d43a591fba407fb862412970667de4,調用api必填參數
1,公務人員數目統計,https://data.gov.mo/Detail?id=06100e7b-a025-44...,按年以季度方式提供澳門特別行政區公共行政人力資源的公務人員數目。不包括受私法制度規範的基金會...,完全開放,行政及法律事務,CSV,行政公職局,（853）88099171,dprtsp@safp.gov.mo,每季,...,"['Ano', 'Trimestre', 'Total']",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,停車場資料,https://data.gov.mo/Detail?id=ac55c2f1-780a-4d...,停車場資料,完全開放,公共交通,XML,交通事務局,88666363,info@dsat.gov.mo,10秒,...,"['-', '-', '-', '-', '-', '-', '-', '-', '-', ...",True,停車場資料,GET,XML,https://dsat.apigateway.data.gov.mo/car_park_d...,停車場資料,Authorization,APPCODE 09d43a591fba407fb862412970667de4,調用api必填參數
3,巴士路線資料,https://data.gov.mo/Detail?id=e7b2e84d-3333-42...,BUS_POLE.SHP---巴士站資料 \nROUTE_NETWORK.SHP---巴士路...,完全開放,公共交通,SHAPEFILEXLS,交通事務局,88666363,info@dsat.gov.mo,按需要,...,"['-', '-', '-', '-', '-', '-', '-', '-', '-', ...",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,實時天氣簡報,https://data.gov.mo/Detail?id=070f7d9b-f734-42...,每小時整點的實況天氣，包括氣溫、濕度、風向及風速等資訊。,完全開放,城市環境,XML,地球物理暨氣象局,(853) 8898 6223,meteo@smg.gov.mo,按需要,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Create clean dataframe

In [67]:
clean_df = df

## Standardizing number formats
#### Converting all the number formats into (853)xxxxxxxx

In [56]:
# function to convert phone numbers into designated format
def format_phone_number(number):
    
    if "；" in number:
        numbers = number.split("；")
        return "; ".join([format_phone_number(n) for n in numbers])
    elif ";" in number:
        numbers = number.split(";")
        return "; ".join([format_phone_number(n) for n in numbers])
    
    raw_number = number.translate({ord(c): None for c in "() +（）"}) # Removes "(", ")", "+" and space
    if len(raw_number) == 11: # Contains international code
        return """({}){}""".format(raw_number[0:3],raw_number[3:])
    elif len(raw_number) == 8: # Doesn't contain international code
        #Assume number is from Macau and add 853
        return """({}){}""".format("853",raw_number)
    else: # Invalid phone number
        error_str = "Invalid number: " + raw_number
        raise Exception(error_str)
        return None

#### Applying function to row

In [70]:
clean_df['提供部門/實體聯絡電話'] = clean_df['提供部門/實體聯絡電話'].apply(format_phone_number)
clean_df.head()

,dataset_name,url,摘要,開放條件,數據分類,數據格式,提供部門/實體,提供部門/實體聯絡電話,提供部門/實體聯絡電郵,更新頻率,...,字段描述（葡萄牙文）,has_api,API名稱,請求方式,返回類型,調用地址,接口描述,api_key,api_val,api_summary
0,酒吧資料,https://data.gov.mo/Detail?id=b4fa6540-4295-4b...,已領有牌照之實體 - 酒吧資料,完全開放,旅遊及博彩,XML,旅遊局,(853)28315566,webmaster@macaotourism.gov.mo,每日,...,"['Nome em T.chinês', 'Nome em S.chinês', 'Nome...",True,dst_bars_list,GET,XML,https://dst.apigateway.data.gov.mo/dst_bars,已領有牌照之實體 - 酒吧 Licensed Entities - Bars 已领有牌照之实...,Authorization,APPCODE 09d43a591fba407fb862412970667de4,調用api必填參數
1,公務人員數目統計,https://data.gov.mo/Detail?id=06100e7b-a025-44...,按年以季度方式提供澳門特別行政區公共行政人力資源的公務人員數目。不包括受私法制度規範的基金會...,完全開放,行政及法律事務,CSV,行政公職局,(853)88099171,dprtsp@safp.gov.mo,每季,...,"['Ano', 'Trimestre', 'Total']",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,停車場資料,https://data.gov.mo/Detail?id=ac55c2f1-780a-4d...,停車場資料,完全開放,公共交通,XML,交通事務局,(853)88666363,info@dsat.gov.mo,10秒,...,"['-', '-', '-', '-', '-', '-', '-', '-', '-', ...",True,停車場資料,GET,XML,https://dsat.apigateway.data.gov.mo/car_park_d...,停車場資料,Authorization,APPCODE 09d43a591fba407fb862412970667de4,調用api必填參數
3,巴士路線資料,https://data.gov.mo/Detail?id=e7b2e84d-3333-42...,BUS_POLE.SHP---巴士站資料 \nROUTE_NETWORK.SHP---巴士路...,完全開放,公共交通,SHAPEFILEXLS,交通事務局,(853)88666363,info@dsat.gov.mo,按需要,...,"['-', '-', '-', '-', '-', '-', '-', '-', '-', ...",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,實時天氣簡報,https://data.gov.mo/Detail?id=070f7d9b-f734-42...,每小時整點的實況天氣，包括氣溫、濕度、風向及風速等資訊。,完全開放,城市環境,XML,地球物理暨氣象局,(853)88986223,meteo@smg.gov.mo,按需要,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,dataset_name,url,摘要,開放條件,數據分類,數據格式,提供部門/實體,提供部門/實體聯絡電話,提供部門/實體聯絡電郵,更新頻率,...,字段描述（葡萄牙文）,has_api,API名稱,請求方式,返回類型,調用地址,接口描述,api_key,api_val,api_summary
0,酒吧資料,https://data.gov.mo/Detail?id=b4fa6540-4295-4b...,已領有牌照之實體 - 酒吧資料,完全開放,旅遊及博彩,XML,旅遊局,(853)28315566,webmaster@macaotourism.gov.mo,每日,...,"['Nome em T.chinês', 'Nome em S.chinês', 'Nome...",True,dst_bars_list,GET,XML,https://dst.apigateway.data.gov.mo/dst_bars,已領有牌照之實體 - 酒吧 Licensed Entities - Bars 已领有牌照之实...,Authorization,APPCODE 09d43a591fba407fb862412970667de4,調用api必填參數
1,公務人員數目統計,https://data.gov.mo/Detail?id=06100e7b-a025-44...,按年以季度方式提供澳門特別行政區公共行政人力資源的公務人員數目。不包括受私法制度規範的基金會...,完全開放,行政及法律事務,CSV,行政公職局,(853)88099171,dprtsp@safp.gov.mo,每季,...,"['Ano', 'Trimestre', 'Total']",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,停車場資料,https://data.gov.mo/Detail?id=ac55c2f1-780a-4d...,停車場資料,完全開放,公共交通,XML,交通事務局,(853)88666363,info@dsat.gov.mo,10秒,...,"['-', '-', '-', '-', '-', '-', '-', '-', '-', ...",True,停車場資料,GET,XML,https://dsat.apigateway.data.gov.mo/car_park_d...,停車場資料,Authorization,APPCODE 09d43a591fba407fb862412970667de4,調用api必填參數
3,巴士路線資料,https://data.gov.mo/Detail?id=e7b2e84d-3333-42...,BUS_POLE.SHP---巴士站資料 \nROUTE_NETWORK.SHP---巴士路...,完全開放,公共交通,SHAPEFILEXLS,交通事務局,(853)88666363,info@dsat.gov.mo,按需要,...,"['-', '-', '-', '-', '-', '-', '-', '-', '-', ...",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,實時天氣簡報,https://data.gov.mo/Detail?id=070f7d9b-f734-42...,每小時整點的實況天氣，包括氣溫、濕度、風向及風速等資訊。,完全開放,城市環境,XML,地球物理暨氣象局,(853)88986223,meteo@smg.gov.mo,按需要,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
print("; ".join(["Hello","There"]))

Hello; There
